In [1]:
import hail as hl

In [2]:
hl.init(default_reference="GRCh38")

Running on Apache Spark version 3.2.3
SparkUI available at http://ip-10-60-4-228.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.116-cd64e0876c94
LOGGING: writing to /opt/notebooks/hail-20240923-0838-0.2.116-cd64e0876c94.log


In [15]:
hl.index_bgen("/tmp/metabolic.bgen", reference_genome="GRCh38")

2024-09-23 08:44:36.324 Hail: INFO: indexed 2394 sites in /tmp/metabolic.bgen at /tmp/metabolic.bgen


In [20]:
d470n = hl.parse_variant("chr17:9889451:G:A")
mt = hl.import_bgen(
    "/tmp/metabolic.bgen",
    entry_fields=["GP"],
    sample_file="/tmp/metabolic.sample",
    variants=d470n,
)

2024-09-23 08:46:36.268 Hail: INFO: Number of BGEN files parsed: 1
2024-09-23 08:46:36.268 Hail: INFO: Number of samples in BGEN files: 465505
2024-09-23 08:46:36.268 Hail: INFO: Number of variants across all BGEN files: 2394
2024-09-23 08:46:36.544 Hail: INFO: Coerced sorted dataset
2024-09-23 08:46:37.144 Hail: INFO: wrote table with 1 row in 1 partition to /tmp/bgen_included_vars-72UtOuXXTaH9N2nseX2Lii.ht
2024-09-23 08:46:38.347 Hail: INFO: Number of BGEN files parsed: 1
2024-09-23 08:46:38.347 Hail: INFO: Number of samples in BGEN files: 465505
2024-09-23 08:46:38.347 Hail: INFO: Number of variants across all BGEN files: 2394


In [21]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'varid': str
----------------------------------------
Entry fields:
    'GP': array<float64>
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [19]:
hl.eval(hl.parse_variant("chr17:9889451:G:A"))

Struct(locus=Locus(contig=chr17, position=9889451, reference_genome=GRCh38), alleles=['G', 'A'])

In [28]:
mt2 = mt.annotate_cols(
    wt=hl.agg.sum(mt.GP[0] == 1),
    het=hl.agg.sum(mt.GP[1] == 1),
    hom=hl.agg.sum(mt.GP[2] == 1),
)

In [45]:
mt2.entries().key_by().drop("s").show(10)

,,,,,,,
locus,alleles,rsid,varid,wt,het,hom,GP
locus<GRCh38>,array<str>,str,str,int64,int64,int64,array<float64>
chr17:9889451,"[""G"",""A""]","""chr17:9889451:G:A""","""chr17:9889451:G:A""",1,0,0,"[1.00e+00,0.00e+00,0.00e+00]"
chr17:9889451,"[""G"",""A""]","""chr17:9889451:G:A""","""chr17:9889451:G:A""",1,0,0,"[1.00e+00,0.00e+00,0.00e+00]"
chr17:9889451,"[""G"",""A""]","""chr17:9889451:G:A""","""chr17:9889451:G:A""",0,1,0,"[0.00e+00,1.00e+00,0.00e+00]"
chr17:9889451,"[""G"",""A""]","""chr17:9889451:G:A""","""chr17:9889451:G:A""",0,0,1,"[0.00e+00,0.00e+00,1.00e+00]"
chr17:9889451,"[""G"",""A""]","""chr17:9889451:G:A""","""chr17:9889451:G:A""",0,1,0,"[0.00e+00,1.00e+00,0.00e+00]"
chr17:9889451,"[""G"",""A""]","""chr17:9889451:G:A""","""chr17:9889451:G:A""",0,0,1,"[0.00e+00,0.00e+00,1.00e+00]"
chr17:9889451,"[""G"",""A""]","""chr17:9889451:G:A""","""chr17:9889451:G:A""",1,0,0,"[1.00e+00,0.00e+00,0.00e+00]"
chr17:9889451,"[""G"",""A""]","""chr17:9889451:G:A""","""chr17:9889451:G:A""",0,1,0,"[0.00e+00,1.00e+00,0.00e+00]"


In [52]:
df = mt2.cols().to_pandas()
df.drop("s", axis=1).value_counts()

2024-09-23 09:07:06.069 Hail: INFO: Coerced sorted dataset


wt  het  hom
1   0    0      218017
0   1    0      198716
    0    1       48508
         0         264
dtype: int64

In [56]:
df2 = df.loc[df.hom.ne(1),]
df2.loc[:, ["s", "s"]].to_csv("no_hom.id", sep="\t", header=False, index=False)

In [57]:
df3 = df.loc[df.hom.eq(1),]
df3.loc[:, ["s", "s"]].to_csv("hom_only.id", sep="\t", header=False, index=False)